In [ ]:
"""
This file is part of mr-recv-coil-match-networks.
Copyright © 2024 Technical University of Denmark (developed by Rasmus Jepsen)

This library is free software; you can redistribute it and/or
modify it under the terms of the GNU Lesser General Public
License as published by the Free Software Foundation; either
version 2.1 of the License, or (at your option) any later version.

This library is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
Lesser General Public License for more details.

You should have received a copy of the GNU Lesser General Public
License along with this library; if not, write to the Free Software
Foundation, Inc., 51 Franklin Street, Fifth Floor, Boston, MA  02110-1301  USA
"""

In [ ]:
"""
This notebook calculates component values for MRI receive coil matching networks and displays circuit diagrams for the calculated networks.
"""

In [ ]:
# import modules
import mr_recv_coil_match_networks.topology
import mr_recv_coil_match_networks.preamp_decoupling
import lc_power_match_baluns.oneport
import numpy as np

In [ ]:
# inputs

# design frequency (Hz)
f0 = 127.73e6

# coil impedance (Ohm)
z_coil = 1.875 + 190.788j

# optimal preamplifier source impedance for noise-matching (Ohm)
z_out = 45.740 + 29.525j

# preamplifier input impedance (Ohm)
z_amp = 45.373 - 135.560j

# the preamplifier decoupling condition to be used, change the class name for different decoupling conditions
# see the mr_recv_coil_match_networks.preamp_decoupling module for more information 
preamp_decoupling = mr_recv_coil_match_networks.preamp_decoupling.HighInputImpedancePreampDecoupling(z_coil, z_out, z_amp)

# The Q-factors below are used to provide performance estimates at the design frequency.
# These performance estimates are only rough and do not account for non-ideal effects other than the Q-factor.
# Capacitors are modelled as series RC circuits and inductors as parallel RL circuits.
# These estimates are not shown in the ideal case where both Q-factors are set to infinity.

# capacitor Q-factor at the design frequency
Qc = 200

# inductor Q-factor at the design frequency
Ql = 15

# minimum noise figure for the preamplifier (dB) (needed for noise figure estimates)
Fmin = 0.408

# noise resistance for the preamplifier (Ohm) (needed for noise figure estimates)
Rn = 2.255

# whether to display TikZ code for circuit diagrams
printTikZ = False

In [ ]:
# draw templates
for cls in mr_recv_coil_match_networks.topology.ReceiveCoilMatchingTopology.__subclasses__():
    if cls.netlist is not None:
        template_circuit = cls.lcapy_circuit()
        print(cls.name)
        template_circuit.draw()
        if printTikZ:
            print(template_circuit.sch._tikz_draw())

In [ ]:
# calculate reactance matrices from [1]
# [1] W. Wang, V. Zhurbenko, J. D. Sánchez‐Heredia, and J. H. Ardenkjær‐Larsen, "Trade‐off between preamplifier noise figure and decoupling in MRI detectors," Magnetic Resonance in Medicine, vol. 89, no. 2, pp. 859–871, 2023. doi:10.1002/mrm.29489 
for i, reactance_parameters in enumerate(preamp_decoupling.reactance_parameters):
    print(f"Solution {i + 1}: (X11, X12, X22) = {reactance_parameters} Ohm")

In [ ]:
# ideal input impedance for matching network (for verification)
print(f"{preamp_decoupling.ideal_zin=} Ohm")

In [ ]:
# ideal output impedance for matching network (for verification)
print(f"{z_out=} Ohm")

In [ ]:
# minimum and maximum values for preamplifier decoupling assuming a lossless matching network (for verification)
print(f"Minimum preamplifier decoupling: {preamp_decoupling.minimum_preamplifier_decoupling} dB")
print(f"Maximum preamplifier decoupling: {preamp_decoupling.maximum_preamplifier_decoupling} dB")

In [ ]:
# output results
for cls_index, cls in enumerate(mr_recv_coil_match_networks.topology.ReceiveCoilMatchingTopology.__subclasses__()):
    print(f"\n{cls.name}")
    for reactance_parameters_index, reactance_parameters in enumerate(preamp_decoupling.reactance_parameters):
        if reactance_parameters_index > 0 and cls.has_only_one_solution:
            break
        print(f"\nSolution {reactance_parameters_index + 1}")
        x11, x12, x22 = reactance_parameters
        topology_reactances = cls.calculate_elements_from_reactance_params(x11, x12, x22)
        components = lc_power_match_baluns.oneport.SimpleLosslessOnePort.from_reactances_at_frequency(topology_reactances, f0)
        if cls.netlist is not None:
            circuit_lc = cls.lcapy_lc_circuit(components)
            circuit_lc.draw()
        for reactance_index, topology_reactance in enumerate(topology_reactances):
            print(f"X{reactance_index + 1} = {topology_reactance} Ohm")
        for component in components:
            print(component)
        if printTikZ:
            print(circuit_lc.sch._tikz_draw())
        if (np.isfinite(Ql) or np.isfinite(Qc)):
            lossy_element_impedances = [component.calculate_impedance(f0, Ql if isinstance(component, lc_power_match_baluns.oneport.Inductor) else Qc) for component in components]
            nf = cls.calculate_noise_figure(z_coil, z_out, Fmin, Rn, lossy_element_impedances)
            rho_in = cls.calculate_rho_in(z_coil, z_amp, lossy_element_impedances)
            decoupling = preamp_decoupling.calculate_preamplifier_decoupling(rho_in)
            cmrr = cls.calculate_cmrr(z_coil, z_amp, lossy_element_impedances)
            print(f"\nPerformance estimates at {f0} Hz:")
            print(f"Noise figure: {nf} dB")
            print(f"Preamplifier decoupling: {decoupling} dB")
            print(f"CMRR: {cmrr} dB")